In [18]:
import extract
import pandas as pd
import os
import geopandas as gpd

In [19]:
import importlib
importlib.reload(extract)

<module 'extract' from '/Users/kguan17/Documents/GitHub/RA-Work/acs/extract.py'>

In [44]:
# open elections.xlsx
elections = pd.read_excel("elections.xlsx")
elections

,filename,city,state,Geography,District #,office,year,geo_filename,tract_filename,place_code
0,NewYorkCity_06222021_DEMBoroughPresidentBronx.csv,New York,NY,boroughcounty,Bronx,Borough president,2021,NaN,NaN,NaN
1,NewYorkCity_06222021_DEMBoroughPresidentKings.csv,New York,NY,boroughcounty,Kings,Borough president,2021,NaN,NaN,NaN
2,NewYorkCity_06222021_DEMBoroughPresidentNewYor...,New York,NY,boroughcounty,New York,Borough president,2021,NaN,NaN,NaN
3,NewYorkCity_06222021_DEMBoroughPresidentQueens...,New York,NY,boroughcounty,Queens,Borough president,2021,NaN,NaN,NaN
4,NewYorkCity_06222021_DEMBoroughPresidentRichmo...,New York,NY,boroughcounty,Richmond,Borough president,2021,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
404,Alaska_11082022_SenateDistrictP.csv,NaN,AK,upper state legislative,P,State senate,2022,alaska_upper_districts_2022.csv,alaska_tracts_2020.csv,NaN
405,Alaska_11082022_SenateDistrictQ.csv,NaN,AK,upper state legislative,Q,State senate,2022,alaska_upper_districts_2022.csv,alaska_tracts_2020.csv,NaN
406,Alaska_11082022_SenateDistrictR.csv,NaN,AK,upper state legislative,R,State senate,2022,alaska_upper_districts_2022.csv,alaska_tracts_2020.csv,NaN
407,Alaska_11082022_SenateDistrictS.csv,NaN,AK,upper state legislative,S,State senate,2022,alaska_upper_districts_2022.csv,alaska_tracts_2020.csv,NaN


In [45]:
# add tracts and percents columns to elections
elections['tracts'] = None
elections['percents'] = None

In [46]:
# getting tracts and percent overlap for each district

# for each row in elections
for index, row in elections.iterrows():
    if index % 10 == 0:
        print(f"Processing row {index}")

    # if geo_filename or tract_filename is not nan
    if not pd.isna(row['geo_filename']) or not pd.isna(row['tract_filename']):
        # if tracts and percents are not already in the row
        if pd.isna(row['tracts']) or pd.isna(row['percents']):
            district = row['District #']

            # open geo_filename and tract_filename in geo_polygons and tract_polygons folders as dfs
            geo_df = pd.read_csv(os.path.join("geo_polygons", row['geo_filename']))
            tract_df = pd.read_csv(os.path.join("tract_polygons", row['tract_filename']))

            # rename geometry columns to WKT
            geo_df = geo_df.rename(columns={"geometry": "WKT"})
            tract_df = tract_df.rename(columns={"geometry": "WKT"})

            # convert dfs to geodataframes
            geo_df = gpd.GeoDataFrame(geo_df, geometry=gpd.GeoSeries.from_wkt(geo_df['WKT']))
            tract_df = gpd.GeoDataFrame(tract_df, geometry=gpd.GeoSeries.from_wkt(tract_df['WKT']))

            # get geometry in geo_df where district is district
            try: 
                district_geo = geo_df[geo_df['district'] == district].geometry.iloc[0]
            except:
                print(f"District {district} not found in {row['geo_filename']}")
                break

            tracts, percents = extract.overlap(tract_df, district_geo)

            # add tracts and percents to elections
            elections.at[index, 'tracts'] = tracts
            elections.at[index, 'percents'] = percents

Processing row 0
Processing row 10
Processing row 20
Processing row 30
Processing row 40
Processing row 50
Processing row 60
Processing row 70
Processing row 80
Processing row 90
Processing row 100
Processing row 110
Processing row 120
Processing row 130
Processing row 140
Processing row 150
Processing row 160
Processing row 170
Processing row 180
Processing row 190
Processing row 200
Processing row 210
Processing row 220
Processing row 230
Processing row 240
Processing row 250
Processing row 260
Processing row 270
Processing row 280
Processing row 290
Processing row 300
Processing row 310
Processing row 320
Processing row 330
Processing row 340
Processing row 350
Processing row 360
Processing row 370
Processing row 380
Processing row 390
Processing row 400


In [47]:
elections.tail(20)

,filename,city,state,Geography,District #,office,year,geo_filename,tract_filename,place_code,tracts,percents
389,Wyoming_04172020_PRESIDENTOFTHEUNITEDSTATES.csv,NaN,WY,state,NaN,President,2020,NaN,NaN,NaN,None,None
390,Alaska_11082022_SenateDistrictA.csv,NaN,AK,upper state legislative,A,State senate,2022,alaska_upper_districts_2022.csv,alaska_tracts_2020.csv,NaN,"[400, 102, 101, 100, 100, 200, 300, 200, 400, ...","[0.9780559068559642, 0.7770836074795211, 0.917..."
391,Alaska_11082022_SenateDistrictC.csv,NaN,AK,upper state legislative,C,State senate,2022,alaska_upper_districts_2022.csv,alaska_tracts_2020.csv,NaN,"[701, 702, 300, 100, 200, 2900, 1000, 800, 900...","[0.24521027938565934, 3.3477238458051567e-09, ..."
392,Alaska_11082022_SenateDistrictD.csv,NaN,AK,upper state legislative,D,State senate,2022,alaska_upper_districts_2022.csv,alaska_tracts_2020.csv,NaN,"[701, 702, 300, 2900, 300, 400, 500, 600, 200,...","[0.0502819701239947, 0.23190249056731935, 6.55..."
393,Alaska_11082022_SenateDistrictE.csv,NaN,AK,upper state legislative,E,State senate,2022,alaska_upper_districts_2022.csv,alaska_tracts_2020.csv,NaN,"[2811, 2813, 300, 2712, 2711, 204, 2812, 2823,...","[0.00011349652241317276, 0.8792924427190709, 0..."
394,Alaska_11082022_SenateDistrictF.csv,NaN,AK,upper state legislative,F,State senate,2022,alaska_upper_districts_2022.csv,alaska_tracts_2020.csv,NaN,"[2602, 2811, 2813, 1732, 2712, 2812, 2823, 260...","[0.9972611340235377, 0.9998865034774218, 0.120..."
395,Alaska_11082022_SenateDistrictG.csv,NaN,AK,upper state legislative,G,State senate,2022,alaska_upper_districts_2022.csv,alaska_tracts_2020.csv,NaN,"[1300, 902, 2712, 2711, 1000, 1401, 1402, 2305...","[7.236544247995435e-06, 0.12174648649118253, 0..."
396,Alaska_11082022_SenateDistrictH.csv,NaN,AK,upper state legislative,H,State senate,2022,alaska_upper_districts_2022.csv,alaska_tracts_2020.csv,NaN,"[1300, 2712, 2711, 1200, 2301, 2714, 2305, 230...","[0.17725223294843345, 0.0014172748034957667, 0..."
397,Alaska_11082022_SenateDistrictI.csv,NaN,AK,upper state legislative,I,State senate,2022,alaska_upper_districts_2022.csv,alaska_tracts_2020.csv,NaN,"[1300, 703, 801, 902, 702, 901, 701, 204, 1200...","[0.1179034518854923, 0.6387621217630376, 0.019..."
398,Alaska_11082022_SenateDistrictJ.csv,NaN,AK,upper state legislative,J,State senate,2022,alaska_upper_districts_2022.csv,alaska_tracts_2020.csv,NaN,"[2602, 801, 802, 902, 901, 701, 980100, 980000...","[0.00273886597646494, 0.4694410172069217, 0.99..."


In [48]:
# save as elections_with_tracts.xlsx
elections.to_excel("elections_with_tracts.xlsx", index=False)